In [1]:
!pip install sentence_transformers

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [1]:
from sentence_transformers import SentenceTransformer , util , models , InputExample, losses ,SentencesDataset, evaluation
from torch.utils.data import DataLoader

In [2]:
import csv
train_samples = []
with open("traindata_sentecesimalarty.tsv", encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t')#, quoting=csv.QUOTE_NONE
    for row in reader:
        sample = InputExample(texts=[row['sentence1'] , row['sentence2']], label=float(round(float(row['simalarty']))))
        train_samples.append(sample)

In [3]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(train_samples,test_size=0.2, shuffle=False)

In [4]:
len(test)

95

In [5]:
word_embedding_model = models.Transformer('deepset/gbert-large')

Some weights of the model checkpoint at deepset/gbert-large were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
tokens = ["<Entity>", "</Entity>"]
word_embedding_model.tokenizer.add_tokens(tokens, special_tokens=True)
word_embedding_model.auto_model.resize_token_embeddings(len(word_embedding_model.tokenizer))

Embedding(31104, 1024)

In [7]:
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
train_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [8]:
train_loss = losses.CosineSimilarityLoss(train_model)

In [9]:
train_dataloader = DataLoader(train, shuffle=True, batch_size=16)

In [10]:
def evalute(test):
    s1 = []
    s2 = []
    l = []
    for i in test:
        s1.append(i.texts[0])
        s2.append(i.texts[1])
        l.append(i.label)
    return evaluation.BinaryClassificationEvaluator(s1,s2,l)   

In [11]:
binary_acc_evaluator = evalute(test)

In [12]:
binary_acc_evaluator = evaluation.BinaryClassificationEvaluator.from_input_examples(test)

In [13]:
import math
warmup_steps = math.ceil(len(train_dataloader) * 10 * 0.1)

In [14]:
train_model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=10, warmup_steps=warmup_steps ,  evaluator=binary_acc_evaluator, evaluation_steps=1000)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/24 [00:00<?, ?it/s]

Iteration:   0%|          | 0/24 [00:00<?, ?it/s]

Iteration:   0%|          | 0/24 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/24 [00:00<?, ?it/s]

Iteration:   0%|          | 0/24 [00:00<?, ?it/s]

Iteration:   0%|          | 0/24 [00:00<?, ?it/s]

Iteration:   0%|          | 0/24 [00:00<?, ?it/s]

Iteration:   0%|          | 0/24 [00:00<?, ?it/s]

In [15]:
binary_acc_evaluator(train_model)

0.9980237154150196

In [16]:
train_model.save('./sentence_simalarity')